In [ ]:
import os 
import torch
import numpy as np

from PIL import Image
from matplotlib import pyplot as plt

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets.folder import default_loader
from torchvision import transforms
import torchvision.models as models

# idea: https://www.kaggle.com/khoatran1312/dog-cat-classification

In [ ]:
# hyperparams

CLASSES = {0:"cat", 1:"dog"}
BATCH_SIZE = 64
IMG_SIZE = (224, 224)
NUM_EPOCHS = 50
TRAIN_DATA_PATH = "../input/dog-vs-cat/train/train"
TEST_DATA_PATH = "../input/dog-vs-cat/test/test"
TRANSFORM_IMG = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMG_SIZE),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

In [ ]:
# custom dataloader for testing samples
class TestingDataset(Dataset):
    
    def __init__(self, root, transforms=None):
        self.transforms = transforms
        self.img_paths = sorted(os.path.join(root, x) for x in os.listdir(root) if x.endswith('.jpg'))

        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = default_loader(img_path)
        if self.transforms is not None:
            sample = self.transforms(sample)
        return sample, img_path.split('/')[-1]

In [ ]:
# data loader
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_data = TestingDataset(root=TEST_DATA_PATH, transforms=TRANSFORM_IMG)
test_data_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg16_bn(pretrained=False, num_classes=2)
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# training
model.train()

for epoch in range(NUM_EPOCHS): 
    running_loss = 0.0
    
    for i, (img,label) in enumerate(train_data_loader):

        inputs, labels = img.to(device), label.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()                                                                                                                                                                                                                                                                                                                                                                                                              

    print('[%d, %5d] loss: %.3f' % (epoch + 1, epoch, running_loss / len(train_data_loader)))
    running_loss = 0.0
print('[INFO]: Finished Training')  


In [ ]:
# PATH = f'./vgg16-plain-{NUM_EPOCHS}.pth'
# torch.save(model.state_dict(), PATH)

In [ ]:
# testing

print('[INFO]: Start Testing')
model.eval()

result_array, image_paths = np.array([]), np.array([])
 
with torch.no_grad():
    for i, (image, image_path) in enumerate(test_data_loader):
        test_input = image.to(device)
        outputs = model(test_input)
        _, predicted = torch.max(outputs, 1)
        result_array = np.append(result_array, predicted.cpu().detach().numpy())
        image_paths = np.append(image_paths, image_path)
                


In [ ]:
result_array = result_array.astype('uint8')
result_array

In [ ]:
# visualization 

for i in range(12):
    plt.subplot(3, 4,i+1)
    image = plt.imread(os.path.join(TEST_DATA_PATH, image_paths[i]))
    plt.imshow(image)
    plt.axis("off")
    title = CLASSES[int(result_array[i])]
    plt.title(f"label: {title}")

In [ ]:
# submission 

import pandas as pd

df = pd.DataFrame({'id': image_paths, 'labels': result_array})
df.to_csv('submission1.csv', index=False)